In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
class Net(nn.Module) :
    def __init__(self,input_size=784,hidden_size=None,output_size=10,activation_type=None):
        super().__init__()
        if hidden_size is None :
            hidden_size = [128, 64]
        self.layers=nn.ModuleList()
        self.layers.append(nn.Linear(input_size,hidden_size[0]))
        for i in range(len(hidden_size)-1):
            self.layers.append(nn.Linear(hidden_size[i],hidden_size[i+1]))
        self.layers.append(nn.Linear(hidden_size[-1],output_size))
        if activation_type is None:
            self.activation = nn.ReLU()
        elif activation_type == "Tanh" :
            self.activation = nn.Tanh()
    def forward(self, x):
        for i in range(len(self.layers)-1):
            x = self.activation(self.layer[i](x))
        return self.layers[-1](x)

model = Net()
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(model.parameters(),lr=0.01,betas=(0.9,.999))
criterion = nn.CrossEntropyLoss() 
print(model)        

Net(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=10, bias=True)
  )
  (activation): ReLU()
)


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.1307,), (0.3081,))
    ])

train_dataset = torchvision.datasets.MNIST(
    root='/data',
    download=True,
    train=True,
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='/data',
    download=True,
    train=False,
    transform=transform
)

train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)


In [ ]:
model.train()
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader :
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}')
        
    

In [ ]:
total = 0
correct = 0

for images,labels in test_loader :
    outputs = model(images)
    correct += torch.sum(outputs == labels)
    total += labels.shape[0]
print(f"Accuracy : {correct/ total : .4f}")